# Required Installation

In [ ]:
!pip install pytorch_pretrained_bert
!pip install transformers
!pip install pyenchant
!apt-get install -y libenchant-dev

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libenchant-dev is already the newest version (1.6.0-11.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


In [ ]:
model = OpenAIGPTDoubleHeadsModel.from_pretrained('openai-gpt')
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')

Some weights of OpenAIGPTDoubleHeadsModel were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['multiple_choice_head.summary.bias', 'lm_head.weight', 'multiple_choice_head.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: ignored

In [ ]:
words = ['<bos>', 'i', 'like', 'playing', 'football', '.', 'i', 'am', 'from', 'NYC', '.', '<speaker1>', 'hello', 'how', 'are', 'you', '?', '<speaker2>', 'i', 'am', 'fine', 'thanks', '.', '<speaker1>', 'great', 'to', 'hear', '<eos>']

In [ ]:
print(words)
words = tokenizer.convert_tokens_to_ids(words)
print(words)

In [ ]:
words_distractor = ['<bos>', 'i', 'like', 'playing', 'football', '.', 'i', 'am', 'from', 'NYC', '.', '<speaker1>', 'hello', 'how', 'are', 'you', '?', '<speaker2>', 'i', 'am', 'fine', 'thanks', '.', '<speaker1>', 'sorry', 'to', 'hear', 'that', '<eos>'] 

In [ ]:
print(words_distractor)
words_distractor = tokenizer.convert_tokens_to_ids(words_distractor)
print(words_distractor)

In [ ]:
# Store the position of the last tokens for the next-sentence prediction loss
last_token = len(words) - 1
last_token_distractor = len(words_distractor) - 1

# Now we can pad reply and distractor inputs and targets to the same length
padding_length = max(len(words), len(words_distractor))
def pad(x, padding):
    return x + [padding] * (padding_length - len(x))

(words, words_distractor) = [pad(x, tokenizer.convert_tokens_to_ids('<pad>'))
                                   for x in (words, words_distractor)]

In [ ]:
print(words,"\n\n",words_distractor)

In [ ]:
input_ids = torch.tensor([[words, words_distractor]], dtype=torch.long)
print(input_ids)
print(input_ids.shape)

In [ ]:
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
model = OpenAIGPTDoubleHeadsModel.from_pretrained('openai-gpt')
tokenizer.add_special_tokens({'cls_token': '[CLS]'})  # Add a [CLS] to the vocabulary (we should train it also!)
model.resize_token_embeddings(len(tokenizer))

choices = ["Hello, my dog is cute [CLS]", "Hello, my cat is cute [CLS]"]
input_ids = torch.tensor([tokenizer.encode(s) for s in choices]).unsqueeze(0)  # Batch size 1, 2 choices
print("input_ids_shape: ",input_ids.shape)
mc_token_ids = torch.tensor([input_ids.size(-1)-1, input_ids.size(-1)-1]).unsqueeze(0)  # Batch size 1
print("mc_token_ids_shape: ",mc_token_ids.shape)

outputs = model(input_ids, mc_token_ids=mc_token_ids)
lm_prediction_scores, mc_prediction_scores = outputs[:2]

#print(lm_prediction_scores,"\n",lm_prediction_scores.shape,"\n",mc_prediction_scores)

In [ ]:
def tokenize(obj):
    if isinstance(obj, str):
        return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(obj))
    if isinstance(obj, dict):
        return dict((n, tokenize(o)) for n, o in obj.items())
    return list(tokenize(o) for o in obj)
        

In [ ]:
url = "https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json"

# Download and load JSON dataset
personachat_file = cached_path(url)
with open(personachat_file, "r", encoding="utf-8") as f:
    dataset = json.loads(f.read())

In [ ]:
train_dataset = dataset['train']
print("size of training set: ",len(train_dataset),"\n")

train_dataset = train_dataset[0]
persona = train_dataset['personality']
print('personality\n')
for s in persona:
  print(s)
print("\n")

utterances = train_dataset['utterances']
print("length of utterances[number of dicts]: ",len(utterances),"\n")
for d in utterances:
  print("\n")
  for key in d:
    print(key,"\n")
    for s in d[key]:
      print(s)
    print("\n")
  print("\n")

    

In [ ]:
train_dataset = dataset["train"]
test_dataset = dataset["valid"]
print(len(train_dataset),", ",len(test_dataset))

In [ ]:
dataset["train"] = dataset["train"][:1]
dataset["valid"] = dataset["valid"][:1]
print(len(dataset["train"])," ",len(dataset["valid"]))

In [ ]:
print(dataset["train"])
dataset = tokenize(dataset)
print(dataset["train"])

In [ ]:
personality_permutations = 1
numOfcandidates = 2
max_history = 2

def pad_dataset(dataset, padding=0):
    """ Pad the dataset. This could be optimized by defining a Dataset class and padding at the batch level, but this is simpler. """
    max_l = max(len(x) for x in dataset["input_ids"])
    for name in PADDED_INPUTS:
        dataset[name] = [x + [padding if name != "lm_labels" else -100] * (max_l - len(x)) for x in dataset[name]]
    return dataset

def build_input_from_segments(persona, history, reply, lm_labels=False, with_eos=True):
    """ Build a sequence of input from 3 segments: persona, history and last reply. """
    bos, eos, speaker1, speaker2 = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[:-1])
    #print(persona)
    sequence =  [[bos] + list(chain(*persona))] + history + [reply + [eos] ]
    # print(sequence)
    sequence = [sequence[0]] + [[speaker2 if (len(sequence)-i) % 2 else speaker1] + s for i, s in enumerate(sequence[1:])]
    # print("sequence: \n",sequence)
    instance = {}
    instance["input_ids"] = list(chain(*sequence))
    instance["token_type_ids"] = [speaker2 if i % 2 else speaker1 for i, s in enumerate(sequence) for _ in s]
    print("token_type_ids:\n", instance['token_type_ids'])
    instance["mc_token_ids"] = len(instance["input_ids"]) - 1
    instance["lm_labels"] = [-100] * len(instance["input_ids"])
    if lm_labels:
        instance["lm_labels"] = ([-100] * sum(len(s) for s in sequence[:-1])) + [-100] + sequence[-1][1:]
    return instance


def get_data_loaders(dataset):
    """ Prepare the dataset for training and evaluation """
    personachat = dataset

    #print("Build inputs and labels")
    datasets = {"train": defaultdict(list), "valid": defaultdict(list)}
    for dataset_name, dataset in personachat.items():
        #print(dataset_name,": len = ",len(dataset))
        num_candidates = len(dataset[0]["utterances"][0]["candidates"])
        #print("num of candidates: ",num_candidates)
        if numOfcandidates > 0 and dataset_name == 'train':
            num_candidates = min(num_candidates, numOfcandidates)
        for dialog in dataset:
            persona = dialog["personality"].copy()
            for _ in range(personality_permutations):
                for utterance in dialog["utterances"]:
                    history = utterance["history"][-(2*max_history+1):]
                    for j, candidate in enumerate(utterance["candidates"][-num_candidates:]):
                        lm_labels = bool(j == num_candidates-1)
                        instance = build_input_from_segments(persona, history, candidate, lm_labels)
                        for input_name, input_array in instance.items():
                            datasets[dataset_name][input_name].append(input_array)
                    datasets[dataset_name]["mc_labels"].append(num_candidates - 1)
                    datasets[dataset_name]["n_candidates"] = num_candidates
                persona = [persona[-1]] + persona[:-1]  # permuted personalities
    
    # for key, val in datasets["train"].items():
    #   print(key)
    #   if isinstance(val, list):
    #     print(len(val))
    print("Pad inputs and convert to Tensor")
    tensor_datasets = {"train": [], "valid": []}
    for dataset_name, dataset in datasets.items():
        print(dataset_name,": ")
        # for key, val in dataset.items():
        #   print(key,": ",type(val))
        dataset = pad_dataset(dataset, padding=tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[-1]))
        for input_name in MODEL_INPUTS:
            # print(input_name,": " ,type(dataset))
            tensor = torch.tensor(dataset[input_name])
            print(input_name,": ",tensor.shape)
            if input_name != "mc_labels":
                tensor = tensor.view((-1, datasets[dataset_name]["n_candidates"]) + tensor.shape[1:])
                print("After reshaping ",input_name,": ",tensor.shape,"\n")
            tensor_datasets[dataset_name].append(tensor)
    return tensor_datasets

    # logger.info("Build train and validation dataloaders")
    # train_dataset, valid_dataset = TensorDataset(*tensor_datasets["train"]), TensorDataset(*tensor_datasets["valid"])
    # train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset) if args.distributed else None
    # valid_sampler = torch.utils.data.distributed.DistributedSampler(valid_dataset) if args.distributed else None
    # train_loader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, shuffle=(not args.distributed))
    # valid_loader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=args.valid_batch_size, shuffle=False)

    # logger.info("Train dataset (Batch, Candidates, Seq length): {}".format(train_dataset.tensors[0].shape))
    # logger.info("Valid dataset (Batch, Candidates, Seq length): {}".format(valid_dataset.tensors[0].shape))
    # return train_loader, valid_loader, train_sampler, valid_sampler

In [ ]:
tensor_dataset = get_data_loaders(dataset)

In [ ]:
for dataset_name, dataset in tensor_dataset.items():
  print(dataset_name,":")
  print(len(dataset))
  for input in dataset:
    print(input.shape)


# My training Code

In [ ]:
'''
The following will be the steps in my training and evaluation process: (the entire process has been distilled down to high-level steps.
                                                                        Each step will be elaborated with corresponding code
                                                                        )
1. I will load raw json data from Google drive
2. Apply some preprocessing to clean the dataset
3. Finally reshape and transform the dataset to feed it to DataLoader.
4. Train for a number of epochs. Evaluate after each epoch.
5. After training is done, I can test the model with some hand-crafted (or freshly fetched) test-data
6. Establish a way of talking to the model - get a continuous conversation going
7. Host the system on a site [not here]
'''


'\nThe following will be the steps in my training and evaluation process: (the entire process has been distilled down to high-level steps.\n                                                                        Each step will be elaborated with corresponding code\n                                                                        )\n1. I will load raw json data from Google drive\n2. Apply some preprocessing to clean the dataset\n3. Finally reshape and transform the dataset to feed it to DataLoader.\n4. Train for a number of epochs. Evaluate after each epoch.\n5. After training is done, I can test the model with some hand-crafted (or freshly fetched) test-data\n6. Establish a way of talking to the model - get a continuous conversation going\n7. Host the system on a site [not here]\n'

In [ ]:
import json 
import torch
import transformers
from google.colab import drive
import os
import datetime
import enchant
import random

import math
import logging
from pprint import pformat
from itertools import chain
from pytorch_pretrained_bert import cached_path

from transformers import OpenAIGPTTokenizerFast
from transformers import OpenAIGPTDoubleHeadsModel
from transformers import GPT2TokenizerFast
from transformers import GPT2DoubleHeadsModel
from transformers import OpenAIGPTLMHeadModel
from transformers import GPT2LMHeadModel
from torch.optim import AdamW
from tokenizers import AddedToken

from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

from tqdm.auto import tqdm
import torch.nn.functional as F

In [ ]:
'''
Mounting Google Drive to this colab instance. Now I should be able to my intended JSON file from drive
'''
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
'''
Here I will initialize and keep the constants
'''

driveRootPath = '/content/gdrive/My Drive/Projects/Conversational AI'
jsonFilePath = '/prangon.json'
modelSavePath = '/model'
punctuationMarks = ['.', '?' ,',', '!', ';', '/', '*', '$', '&', '(', ')' ,'[', ']', '%','"',"'",':','-','=','+', '_', '>', '<']
permittedEmoji = [':3']
specialTokens = ['<speaker1>','<speaker2>','<bos>','<eos>','<pad>']
ATTR_TO_SPECIAL_TOKEN = {'bos_token': '<bos>', 'eos_token': '<eos>', 'pad_token': '<pad>',
                         'additional_special_tokens': ['<speaker1>', '<speaker2>']}

MODEL_INPUTS = ['input_ids', 'token_type_ids', 'lm_labels', 'mc_token_ids', 'mc_labels']
PADDED_INPUTS = [input_type for input_type in MODEL_INPUTS if 'mc' not in input_type]
# print(PADDED_INPUTS)

model_type = 'openai-gpt'
pairOfExchanges = 3

In [ ]:
rawDataJson =  open(os.path.normpath(driveRootPath+jsonFilePath))
messageFile = json.load(rawDataJson)

# Utility Functions

In [ ]:
"""
Utility Functions
"""

def readLists(dictName, dictKey):
    print(dictKey,":")
    for item in dictName[dictKey]:
        print(item)
    print("\n")

def groupMessagesByDate(messageList):
    returnDict = {}
    for message in messageList:
        if message['type'] == "Generic" and 'content' in message:
            timeStamp = datetime.datetime.fromtimestamp(message['timestamp_ms']/1000.0).strftime('%d/%m/%Y')
            #print(message)
            if timeStamp in returnDict:
                returnDict[timeStamp].append({'content' : message["content"], 'sender_name':message['sender_name']})
            else:
                returnDict[timeStamp] = [{'content' : message["content"], 'sender_name':message['sender_name']}]

    return returnDict

def groupMessagesBySentenceNum(messageList):
    returnDict = []
    currentList = []
    for message in messageList:
      if len(currentList) == 2*pairOfExchanges:
        returnDict.append(currentList)
        currentList = []
      if message['type'] == "Generic" and 'content' in message:
          currentList.append({'content' : message["content"], 'sender_name':message['sender_name']})
    return returnDict

def filterGroupMessages(groupedMessage, otherName):
    '''this function will only keep the message exchanges that are fit for training'''

    '''First off, we will convert each conversation so that it begins with my texts'''

    for conversationIdx, messages in enumerate(groupedMessage):
        startIdx = -1
        for idx, message in enumerate(messages):
            #print(message)
            if otherName not in message['sender_name']: 
                startIdx = idx
                break
 
        groupedMessage[conversationIdx] = messages[startIdx:]

    '''Second, we are gonna concatenate consecutive messages from one sender and coalesce them into one message'''
    for conversationIdx, messages in enumerate(groupedMessage):
        if len(messages)>0:
            newMessages = []
            currentIdx = 1
            prevSender = messages[0]["sender_name"]
            prevContent = messages[0]["content"]

            while currentIdx < len(messages):
                if messages[currentIdx]["sender_name"] == prevSender:
                    prevContent += (" "+ messages[currentIdx]["content"])
                else:
                    newMessages.append({'sender_name':prevSender,'content': prevContent})
                    prevContent = messages[currentIdx]["content"]
                    prevSender = messages[currentIdx]["sender_name"]
                currentIdx += 1
            newMessages.append({'sender_name':prevSender,'content': prevContent})
            groupedMessage[conversationIdx] = newMessages

    '''Third, we will convert the conversations so that they end with the other person's sentence. In this way
        we are ensuring that there will be even number of exchanges in each conversation. Furthermore, in this way
        every exchange will begin with my text and end with the robot's reply
    '''
    
    for conversationIdx, messages in enumerate(groupedMessage):
        if len(messages) > 0 and otherName not in messages[len(messages)-1]['sender_name']:
            groupedMessage[conversationIdx] = messages[:-1]

    '''Fourth, we will remove all the conversations that include fewer than one exchange from each party.
    Moreover, we will convert the dictionary to a list of conversations (which themselves will be lists of strings)'''
    conversationList = [messages for messages in groupedMessage if len(messages)>1]
    conversationList = [ [[message['content'] for message in messages ]] for messages in conversationList ]


    ''' Now we will breed more conversations from those we have :3 
        Conversations that have more than 2 exchanges have been broken down to make more conversations.
        Conversations that have more than 2 exchanges have been split to make conversations with 4, 6,... exchanges.
    '''
    # conversationCount = len(conversationList)
    # for idx in range(conversationCount):
    #     conversation = conversationList[idx][0]
    #     conversationLength = len(conversation)

    #     if conversationLength > 2:
    #         endIdx = 2
    #         while endIdx < conversationLength:
    #             conversationList.append([conversation[:endIdx]])
    #             endIdx += 2

    # print("conversation count: ",len(conversationList))
    # messagesCount2(conversationList, "Naimul")
    
    
    # for idx in range(5):
    #     print(conversationList[idx],"\n\n",conversationList[idx+212])

    return conversationList
def removeNewLines(conversationList):
    
    newConversationList = [[[sentence.replace("\n", " ") for sentence in conversation[0]]] for conversation in conversationList]
    return newConversationList

'''places space before every punctuation mark and numbers such as ?, ., ',', !, /, *, $, &, (, ), [, ], %   '''
def spaceBeforePunctuation_Numbers(conversationList):
    punctuations = punctuationMarks + [str(i) for i in range(10)]

    for punctuation in punctuations:
        conversationList = [[[sentence.replace(punctuation, " "+punctuation+" ") for sentence in conversation[0]]] for conversation in conversationList]

    return conversationList

""" Emojis appear as unicode characters that are not alphanumeric. 
    At times they appear as hexadecimal code.
    So appear such characters or codes. We do this in order to keep 
    only Bengali words written with English letters inside nonEnglishWords list
"""
def removeEmoji(conversationList):
    
    for conversationIdx in range(len(conversationList)):
        conversation = conversationList[conversationIdx][0]

        for sentenceIdx in range(len(conversation)):
            sentence = conversation[sentenceIdx]
            newSentence = ""
            words = sentence.split(" ")
            for word in words:
                if word in punctuationMarks or word.isalnum():
                    newSentence += (word+" ")
            conversationList[conversationIdx][0][sentenceIdx] = newSentence
            
    return conversationList

def findNonEnglishWords(conversationList):
    uniqueWords = set()
    nonEnglishWords = []
    wordChecker = enchant.Dict("en_US")

    for conversation in conversationList:
        for sentence in conversation[0]:
            for word in sentence.split(" "):
                if len(word) > 0:
                    uniqueWords.add(word)
    for word in uniqueWords:
        if wordChecker.check(word) == False:
            nonEnglishWords.append(word)

    return nonEnglishWords
################################# USE THIS LATER IF RESPONSES BECOME TOO MEANINGLESS ################
def removingEmptySentence(conversationList):

  """
  This function will remove sentences that have no words or only have punctuation marks and nothing else
  After doing this, we will again filter out conversation with fewer than two exchanges
  """

  conversationList = [[conversation for conversation in choices] for choices in conversationList]


In [ ]:
for key in messageFile:
    if not isinstance(messageFile[key],list):
        print(key,": ",messageFile[key])
    else:
        print(key,": (length of list) ",len(messageFile[key]))
    print("\n")

for key in messageFile:
    if key != 'messages' and isinstance(messageFile[key],list):
        readLists(messageFile, key)


participants : (length of list)  2


messages : (length of list)  7069


title :  Naimul Prangon


is_still_participant :  True


thread_type :  Regular


thread_path :  inbox/naimulprangon_2yp3tjb4fq


magic_words : (length of list)  0


participants :
{'name': 'Naimul Prangon'}
{'name': 'Syed Zami Ul-Haque Navid'}


magic_words :




# Pre-Processing

In [ ]:
#sort messages in the increasing order of timestamp
list.reverse(messageFile["messages"])

In [ ]:
"""
The current grouping-by-date system makes some conversation arbitrarily longer. As a result, input_ids have to contain over 1400 tokens. 
I have to make the conversations shorter so that the input_ids are able to contain 512 or fewer tokens.
For now each group will have 4 pairs of exchanges - and hopefully 8 sentences in total
"""
# groupedMessages = groupMessagesByDate(messageFile["messages"])
# for date in groupedMessages:
#     print(date,": ",len(groupedMessages[date]))

# print(groupedMessages['10/09/2021'])

groupedMessages = groupMessagesBySentenceNum(messageFile["messages"])
print(len(groupedMessages))

for message in groupedMessages[0]:
  print(message)


1140
{'content': 'howard batman', 'sender_name': 'Syed Zami Ul-Haque Navid'}
{'content': "what's this", 'sender_name': 'Naimul Prangon'}
{'content': 'dongle', 'sender_name': 'Syed Zami Ul-Haque Navid'}
{'content': 'in a hat', 'sender_name': 'Syed Zami Ul-Haque Navid'}
{'content': 'oh', 'sender_name': 'Naimul Prangon'}
{'content': 'tor nephew er jonne gift kinli?', 'sender_name': 'Naimul Prangon'}


In [ ]:
conversationList = filterGroupMessages(groupedMessages, 'Naimul')
conversationList = removeNewLines(conversationList)
conversationList = spaceBeforePunctuation_Numbers(conversationList)
conversationList = removeEmoji(conversationList)
"""
replace empty responses with 'wut' emojis 
"""
for conversationIdx, conversation in enumerate(conversationList):
  for choiceIdx, choice in enumerate(conversation):
    for messageIdx, message in enumerate(choice):
      if len(message) < 2:
        conversationList[conversationIdx][choiceIdx][messageIdx] = 'wut'

################################# USE THIS LATER IF RESPONSES BECOME TOO MEANINGLESS ################
# conversationList = removingEmptySentence(conversationList):
nonEnglishWords = findNonEnglishWords(conversationList)

In [ ]:
"""
Include a distractor with every sentence
"""
random.shuffle(conversationList)
for conversationIdx in range(len(conversationList)):
  distractorConversation = conversationList[conversationIdx][0][:-1] \
                            + [conversationList[ (conversationIdx+1) % len(conversationList) ][0][-1]]
  conversationList[conversationIdx].append(distractorConversation)

In [ ]:
"""
sanity check
"""
for conversationIdx in range(5,7):
    print(conversationList[conversationIdx])
print(nonEnglishWords)

In [ ]:
"""
split dataset into train and validation
"""
print(len(conversationList))
# random.shuffle(conversationList)
datasets = {'train':conversationList[:900], 'valid':conversationList[900:]}
print(len(datasets['train']), " ", len(datasets['valid']))

1050
900   150


In [ ]:
"""
sanity check
"""
for dataset_name, dataset in datasets.items():
  print(dataset_name)
  for choices in dataset:
    for conversation in choices:
      print(conversation)
    break

train
['nice chobi de : 3 ', 'kisher ', 'gari thelar ', 'wut']
['nice chobi de : 3 ', 'kisher ', 'gari thelar ', 'eita to mone hoy recently shesh hoise ']
valid
['ikr ', 'but yeah proper way ', 'scared er pashapashi ei karoneo procrastinate kori ', 'oneke generic mail dey ']
['ikr ', 'but yeah proper way ', 'scared er pashapashi ei karoneo procrastinate kori ', 'first e ki rakhsilen ? ']


# Dataset & model preparation - Training & Decoding

In [ ]:
"""
download, load model & tokenizers
"""

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer_class = OpenAIGPTTokenizerFast if model_type == "openai-gpt" else GPT2TokenizerFast
tokenizer = tokenizer_class.from_pretrained(model_type)

# model_class = OpenAIGPTDoubleHeadsModel if model_type == "openai-gpt" else GPT2DoubleHeadsModel
# model = model_class.from_pretrained(model_type).to(device)

Downloading:   0%|          | 0.00/816k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/458k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/656 [00:00<?, ?B/s]

In [ ]:
print(len(tokenizer))
"""
Adding new tokens to the tokenizer
"""
tokenizer.add_tokens([AddedToken(banglish, lstrip = True, rstrip = True) for banglish in nonEnglishWords])

"""
Adding special tokens to the tokenizer
"""
tokenizer.add_special_tokens(ATTR_TO_SPECIAL_TOKEN)
# model.resize_token_embeddings(len(tokenizer))
print(len(tokenizer))

40478
42824


In [ ]:
"""
Now Tokenize the dataset
"""
# test_sentence = "This is a test sentence . right ?"
# print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(test_sentence)))
# print(tokenizer.encode(test_sentence))
# print(tokenizer.decode([616, 544, 246, 2345, 5958, 239, 255, 1358, 4, 241, 257])," ",tokenizer.decode([616, 544, 246, 2345, 5958, 239, 255, 41066, 4, 241, 257]))

def tokenize(tokenizer, token):

  if isinstance(token, str):
    return tokenizer.encode(token)
  if isinstance(token, dict):
    return dict((key, tokenize(tokenizer,val)) for key, val in token.items())
  return list( tokenize(tokenizer, item) for item in token)

# test_dataset = {'train':[datasets["train"][0]], 'test':[datasets["valid"][0]]}
# for dataset_name, dataset in test_dataset.items():
#   print(dataset_name)
#   for choices in dataset:
#     print(len(choices))
#     for conversation in choices:
#       print(conversation,"\n")
#     break
# test_dataset = tokenize(tokenizer,test_dataset)

# for dataset_name, dataset in test_dataset.items():
#   print(dataset_name)
#   for choices in dataset:
#     print(len(choices))
#     for conversation in choices:
#       print(conversation,"\n")
#     break



# Dataset & Model Preparation - Training

In [ ]:
datasets = tokenize(tokenizer, datasets)
# test_dataset = tokenize(tokenizer, test_dataset)

In [ ]:
########### SANITY CHECK #############

for dataset_name, dataset in datasets.items():
  print(dataset_name,": ",len(dataset))
  for choices in dataset:
    print(len(choices))
    for conversation in choices:
      print(conversation,"\n")
    break

# print(tokenizer.convert_tokens_to_ids(specialTokens[:-1]))

train :  900
2
[[40595, 697, 590, 40873, 514, 249, 1064, 6224, 42195, 507, 256, 252, 481, 2068, 697, 590, 40873, 514, 249, 825, 249, 1048, 7400, 249, 42724, 485, 5163, 258, 40904, 10019], [507, 3793, 239, 26553, 246, 41721, 20915, 3396, 40728, 41287, 42439, 795, 239, 507, 1598, 1150, 239]] 

[[40595, 697, 590, 40873, 514, 249, 1064, 6224, 42195, 507, 256, 252, 481, 2068, 697, 590, 40873, 514, 249, 825, 249, 1048, 7400, 249, 42724, 485, 5163, 258, 40904, 10019], [9085, 485, 704, 3165, 2057, 5687, 40613, 257]] 

valid :  150
2
[[271, 281, 41772, 40836, 41867, 42017, 246, 6494], [14433, 3317, 13810, 41867, 6319, 42488, 264, 17556, 41368, 41347, 257], [32821, 239, 7182, 249, 483, 41357, 42282, 41571], [42707, 253, 239, 41369, 2066]] 

[[271, 281, 41772, 40836, 41867, 42017, 246, 6494], [14433, 3317, 13810, 41867, 6319, 42488, 264, 17556, 41368, 41347, 257], [32821, 239, 7182, 249, 483, 41357, 42282, 41571], [40697, 483, 7188]] 



In [ ]:
"""
1. Now I would insert the bos, speaker1/2, eos tokens into the conversations to discern the beginning and end of each segment of a dialogue. 

2. Moreover I will convert a list of strings into a list of words (words that make up the list of strings). These list of words will be the inputs
to the model (after tokenization)

"""

speaker1, speaker2, bos, eos = tokenizer.convert_tokens_to_ids(specialTokens[:-1])

def joinConversationSentences(datasets):
  for dataset_name, dataset in datasets.items():
    conversationArray = []
    for conversationChoices in dataset:
      # print(len(conversationChoices),", ",conversationChoices)
      # break
      choicesList = []
      for conversation in conversationChoices:
        wordList = [bos]
        for sentenceIdx in range(len(conversation)):
          sentence = conversation[sentenceIdx]
          # print(sentence)
          wordList += [speaker2 if sentenceIdx%2 else speaker1]
          wordList += sentence
        wordList += [eos]
        choicesList.append(wordList)
      conversationArray.append(choicesList)
    datasets[dataset_name] = conversationArray
  return datasets

In [ ]:
datasets = joinConversationSentences(datasets)

In [ ]:
########## SANITY CHECK #############

for dataset_name, dataset in datasets.items():
  print(dataset_name,": ",len(dataset))
  for choices in dataset:
    print(len(choices))
    for conversation in choices:
      print(conversation,"\n")
    break

train :  900
2
[42819, 42822, 40595, 697, 590, 40873, 514, 249, 1064, 6224, 42195, 507, 256, 252, 481, 2068, 697, 590, 40873, 514, 249, 825, 249, 1048, 7400, 249, 42724, 485, 5163, 258, 40904, 10019, 42823, 507, 3793, 239, 26553, 246, 41721, 20915, 3396, 40728, 41287, 42439, 795, 239, 507, 1598, 1150, 239, 42820] 

[42819, 42822, 40595, 697, 590, 40873, 514, 249, 1064, 6224, 42195, 507, 256, 252, 481, 2068, 697, 590, 40873, 514, 249, 825, 249, 1048, 7400, 249, 42724, 485, 5163, 258, 40904, 10019, 42823, 9085, 485, 704, 3165, 2057, 5687, 40613, 257, 42820] 

valid :  150
2
[42819, 42822, 271, 281, 41772, 40836, 41867, 42017, 246, 6494, 42823, 14433, 3317, 13810, 41867, 6319, 42488, 264, 17556, 41368, 41347, 257, 42822, 32821, 239, 7182, 249, 483, 41357, 42282, 41571, 42823, 42707, 253, 239, 41369, 2066, 42820] 

[42819, 42822, 271, 281, 41772, 40836, 41867, 42017, 246, 6494, 42823, 14433, 3317, 13810, 41867, 6319, 42488, 264, 17556, 41368, 41347, 257, 42822, 32821, 239, 7182, 249, 483, 

In [ ]:
for dataset_name, dataset in datasets.items():
  datasets[dataset_name] = {}
  datasets[dataset_name]['input_ids'] = dataset
  # print(dataset_name,":\n",datasets[dataset_name]['input_ids'][0],"\n",datasets[dataset_name]['input_ids'][1])

In [ ]:
"""
sanity check
"""

for dataset_name, dataset in datasets.items():
  print(dataset_name,": ")
  for conversationChoices in dataset['input_ids']:
    for conversation in conversationChoices:
      print(conversation)
    break

train : 
[42819, 42822, 40595, 697, 590, 40873, 514, 249, 1064, 6224, 42195, 507, 256, 252, 481, 2068, 697, 590, 40873, 514, 249, 825, 249, 1048, 7400, 249, 42724, 485, 5163, 258, 40904, 10019, 42823, 507, 3793, 239, 26553, 246, 41721, 20915, 3396, 40728, 41287, 42439, 795, 239, 507, 1598, 1150, 239, 42820]
[42819, 42822, 40595, 697, 590, 40873, 514, 249, 1064, 6224, 42195, 507, 256, 252, 481, 2068, 697, 590, 40873, 514, 249, 825, 249, 1048, 7400, 249, 42724, 485, 5163, 258, 40904, 10019, 42823, 9085, 485, 704, 3165, 2057, 5687, 40613, 257, 42820]
valid : 
[42819, 42822, 271, 281, 41772, 40836, 41867, 42017, 246, 6494, 42823, 14433, 3317, 13810, 41867, 6319, 42488, 264, 17556, 41368, 41347, 257, 42822, 32821, 239, 7182, 249, 483, 41357, 42282, 41571, 42823, 42707, 253, 239, 41369, 2066, 42820]
[42819, 42822, 271, 281, 41772, 40836, 41867, 42017, 246, 6494, 42823, 14433, 3317, 13810, 41867, 6319, 42488, 264, 17556, 41368, 41347, 257, 42822, 32821, 239, 7182, 249, 483, 41357, 42282, 4157

In [ ]:
"""
So far I have created inputs_ids of the dataset
The dataset is now a dictionary. The following are dimenions.
Datasets => [key: 'train' or 'valid'] [key: 'input_ids'] [list of length 1500 for train or 113 for valid] [2 choices = reply & distractor] [variable length of list of IDs of tokenized words ]

Next I will be creating a function that creates
1. segment identifiers for each conversation using inputs_ids of each conversation. Dimension will be the same as input_ids
2. lm_labels using input_ids of each conversation. Dimension will be the same as inputs_ids
3. mc_token_ids using length of each conversation. Dimensions will be [key: 'mc_token_ids'][list of length 1500 for train or 113 for valid] [2 choices = reply & distractor]
4. mc_labels using some of the dimensions of the input_ids. Dimensions will be [key: 'mc_token_ids'][list of length 1500 for train or 113 for valid]
"""

"\nSo far I have created inputs_ids of the dataset\nThe dataset is now a dictionary. The following are dimenions.\nDatasets => [key: 'train' or 'valid'] [key: 'input_ids'] [list of length 1500 for train or 113 for valid] [2 choices = reply & distractor] [variable length of list of IDs of tokenized words ]\n\nNext I will be creating a function that creates\n1. segment identifiers for each conversation using inputs_ids of each conversation. Dimension will be the same as input_ids\n2. lm_labels using input_ids of each conversation. Dimension will be the same as inputs_ids\n3. mc_token_ids using length of each conversation. Dimensions will be [key: 'mc_token_ids'][list of length 1500 for train or 113 for valid] [2 choices = reply & distractor]\n4. mc_labels using some of the dimensions of the input_ids. Dimensions will be [key: 'mc_token_ids'][list of length 1500 for train or 113 for valid]\n"

In [ ]:
def createDataset(conversationChoice):
  """
  First we will create segment identifiers
  Then, lm_labels
  """

  segmentIds = []
  lm_labels = []
  replyStartIdx = -1
  mc_token_ids = [min(511,len(conversation)-1) for conversation in conversationChoice]
  #[len(conversation)-1 for conversation in conversationChoice]
  mc_labels = 0 #0th index is the real reply, idx 1 contains the distractor

  for conversationIdx, conversation in enumerate(conversationChoice):
    segmentsForConversation = []
    prevSegment, currentSegment = speaker2, speaker1 #<speaker1>

    for wordIdx, word in enumerate(conversation):
      if word == speaker2:
        replyStartIdx = wordIdx
      if word == prevSegment:
        temp = currentSegment
        currentSegment = prevSegment
        prevSegment = temp
      segmentsForConversation.append(currentSegment)
    segmentIds.append(segmentsForConversation)

    lm_labels.append([-100]*(len(conversation)))
    if conversationIdx == 0: #this is the real reply, idx 1 contains the distractor
      """
      We should not include <speaker2> token in the label. Hence, it should be replyStartIdx + 1 in the following line. Because replyStartIdx holds the position of the last <speaker2> token
      """
      lm_labels[conversationIdx][replyStartIdx+1:] = conversation[replyStartIdx+1:]
  
  # for segment in segmentIds:
  #   print(segment)
  # for label in lm_labels:
  #   print(label)
  # print(mc_token_ids,"\n",mc_labels)

  return (segmentIds, lm_labels, mc_token_ids, mc_labels)
  

In [ ]:
for dataset_name, dataset in datasets.items():
  print(dataset_name,":")
  
  for input_name in MODEL_INPUTS:
    if input_name != 'input_ids':        
      datasets[dataset_name][input_name] = []

  for conversationChoice in dataset['input_ids']:
    token_type_ids, lm_labels, mc_token_ids, mc_labels = createDataset(conversationChoice)
    datasets[dataset_name]['token_type_ids'].append(token_type_ids)
    datasets[dataset_name]['lm_labels'].append(lm_labels)
    datasets[dataset_name]['mc_token_ids'].append(mc_token_ids)
    datasets[dataset_name]['mc_labels'].append(mc_labels)

train :
valid :


In [ ]:
def sizeOfNestedList(listName, sizeStr):
  innerItem = listName[0]
  if isinstance(innerItem, list):
    sizeStr += (" "+str(len(listName)))
    return sizeOfNestedList(innerItem, sizeStr)
  
  print(sizeStr+" "+str(len(listName)))
  

In [ ]:
######## SANITY CHECK ############
for dataset_name in datasets:
  print(dataset_name,": ")
  for key in datasets[dataset_name]:
    print(key)
    sizeOfNestedList(datasets[dataset_name][key], "")
  print("\n\n")

train : 
input_ids
 900 2 51
token_type_ids
 900 2 51
lm_labels
 900 2 51
mc_token_ids
 900 2
mc_labels
 900



valid : 
input_ids
 150 2 38
token_type_ids
 150 2 38
lm_labels
 150 2 38
mc_token_ids
 150 2
mc_labels
 150





In [ ]:
"""
create ml_labels, mc_token_id, mc_labels
consolidate creation of all the inputs except the input_ids into one function
OH FUCK I FORGOT TO TOKENIZE
I GOTTA TOKENIZE BEFORE I GO ANY FURTHER
Before tokenization I will have to add the banglish tokens and the special tokens to the tokenizer and the model
"""

'\ncreate ml_labels, mc_token_id, mc_labels\nconsolidate creation of all the inputs except the input_ids into one function\nOH FUCK I FORGOT TO TOKENIZE\nI GOTTA TOKENIZE BEFORE I GO ANY FURTHER\nBefore tokenization I will have to add the banglish tokens and the special tokens to the tokenizer and the model\n'

# Readying input & model for training

In [ ]:
"""
Now the dataset has been created as lists & dictionaries
Next I'll be
1. padding the relevant portions of the dataset
2. turn the dataset into tensors
3. One iteration for model training
4. Inference model
5. Decoding model
"""

"\nNow the dataset has been created as lists & dictionaries\nNext I'll be\n1. padding the relevant portions of the dataset\n2. turn the dataset into tensors\n3. One iteration for model training\n4. Inference model\n5. Decoding model\n"

In [ ]:
def padDataset(dataset, datasets, input_name, dataset_name):
  max_len = -1
  for conversationChoices in dataset:
      for conversation in conversationChoices:
        max_len = max(max_len, len(conversation))
  print(max_len) 


  for choiceIdx, conversationChoices in enumerate(dataset):
      for convIdx, conversation in enumerate(conversationChoices):
        paddingArray = tokenizer.encode(specialTokens[-1]) if input_name != 'lm_labels' else [-100]
        datasets[dataset_name][input_name][choiceIdx][convIdx] += ( paddingArray * (max_len - len(conversation)))


In [ ]:
for dataset_name, dataset in datasets.items():
  for input_name in PADDED_INPUTS:
    print(input_name)
    padDataset(dataset[input_name], datasets, input_name, dataset_name)

input_ids
428
token_type_ids
428
lm_labels
428
input_ids
239
token_type_ids
239
lm_labels
239


In [ ]:
"""
removing the last sentences from valid dataset, to see if we can get better at creating legit replies
"""
if False:
  print("here")
  speaker2 = tokenizer.encode('<speaker2>')[0]
  padding = tokenizer.encode(specialTokens[-1])[0]
  # print(speaker2)
  # lastSpeaker2Idx = -1

  for choiceIdx, choicesList in enumerate(datasets['valid']['input_ids']):
    for conversationIdx, conversation in enumerate(choicesList):
      conversationLen = len(conversation)
      for idx in range(conversationLen-1, 0, -1):
        if conversation[idx] == speaker2:
          # print(choiceIdx,", ",conversationIdx)
          # lastSpeaker2Idx = idx
          datasets['valid']['input_ids'][choiceIdx][conversationIdx] = datasets['valid']['input_ids'][choiceIdx][conversationIdx][:(idx+3)] + [padding]*(conversationLen - idx-3)
          datasets['valid']['token_type_ids'][choiceIdx][conversationIdx] = datasets['valid']['token_type_ids'][choiceIdx][conversationIdx][:(idx+3)] + [padding]*(conversationLen - idx-3)
          break
      


In [ ]:
"""
Creating Dataset and DataLoader
Since I only have an iterable-type [and not map-like] dataset, I do not need a sampler. Batch-size and shuffle would suffice
Since I do not have access to neither multiple GPU nor am I too much worried about speeding up the process, I do not need to use
multiple workers for Data loading. Also, I do not need to think about distributed or parallel training
"""

trainDataset = TensorDataset( *[torch.tensor(input).to(device) for _,input in datasets['train'].items()] )
validDataset = TensorDataset(*[torch.tensor(input).to(device) for _,input in datasets['valid'].items()])

trainDataLoader = DataLoader(trainDataset, batch_size = 4, shuffle = True)
validDataLoader = DataLoader(validDataset, batch_size = 4)

In [ ]:
######### SANITY CHECK ############
for batch in trainDataLoader:
  input_ids, token_type_ids, lm_labels, mc_token_ids, mc_labels = batch
  print(input_ids.shape," ",input_ids.device," ", token_type_ids.shape," ", lm_labels.shape, " ", mc_token_ids.shape, " ",mc_labels.shape)
  break

torch.Size([4, 2, 428])   cuda:0   torch.Size([4, 2, 428])   torch.Size([4, 2, 428])   torch.Size([4, 2])   torch.Size([4])


In [ ]:
"""
########### Model hyperparameters declaration ############
"""
initLR = 6.25e-5
epochs = 25
num_trainingSteps = epochs * len(trainDataLoader)
train_loss = {'lm_loss':[], 'mc_loss':[], 'total_loss':[]}
lm_loss_coeff, mc_loss_coeff = 1, 1

In [ ]:
optimizer = AdamW(model.parameters(), lr=initLR)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer=optimizer, num_warmup_steps=0,num_training_steps=num_trainingSteps)

In [ ]:
import torch.nn.functional as F
def findCELoss(lm_logits, lm_labels):
  ######### lm_logits is a 4 dimenstional tensor - (batch_size, number_of_options, sequence_length, vocab_size) - this contains softmax probability #########
  ######## lm_labels is a 3 dimensional tensor - (batch_size, number_of_options, sequence_length) ############
  """
  1. hot encode the labels. Don't have to. The CrossEntropy function takes care of it
  2. flatten both tensors
  3. compute CE loss
  """
  batchSize = lm_labels.shape[0] #* lm_labels.shape[1]
  lm_labels = lm_labels[...,1:].contiguous().view(-1)
  lm_logits = lm_logits[...,1:,:].contiguous().view(-1, lm_logits.shape[-1])

  Lce = torch.nn.CrossEntropyLoss(ignore_index = -100)
  loss = Lce(input=lm_logits, target=lm_labels) / batchSize

  return loss

def findPrecisionAndRecall(mc_logits, mc_labels, ):
  ######## mc_logits is of dimensions (batch_size, num_of_options) ########
  ######## mc_labels is of dimensions (batch_size) ###########

  """
  1. apply argmax on mc_logits
  2. apply condtions to count TP, TN, FP, FN
  3. return Recall and Precision
  """
  recall = 0
  precision = 0
  ### to avoid division by zero #####
  TP = 1e-8
  TN = 0
  FN = 1e-8
  FP = 1e-8

  batchSize = mc_labels.shape[0]

  mc_logits = torch.argmax(mc_logits, dim = -1)

  for idx, label in enumerate(mc_labels):
    if label == 1:
      if mc_logits[idx] == 1:
        TP += 1
      else:
        FN += 1
    elif label == 0:
      if mc_logits[idx] == 1:
        FP += 1
      else:
        TN += 1

  TP, TN = TN, TP
  FP, FN = FN, FP

  recall = TP/(TP+FN)
  precision = TP/(TP+FP)

  return precision, recall

In [ ]:
if False:
  logits = torch.rand(4, 2, 5, 10)
  softmax = torch.nn.Softmax(dim=-1)
  logits = softmax(logits)
  labels = torch.Tensor(4,2,5).random_(0,9).to(torch.int64)
  print("CE loss: ", findCELoss(logits, labels))

  mc_logits = torch.rand(4,2)
  mc_logits = softmax(mc_logits)
  mc_labels = torch.zeros(4)
  print(mc_labels)

  print("precision, recall: ", findPrecisionAndRecall(mc_logits, mc_labels))

In [ ]:
def trainingStep(model, optimizer, dataLoader, progressbar):
  train_loss = {'lm_loss':0, 'mc_loss':0, 'total_loss': 0}

  model.train()
  for batch in dataLoader:
    input_ids, token_type_ids, lm_labels, mc_token_ids, mc_labels = batch
    outputs = model(input_ids = input_ids, token_type_ids = token_type_ids, labels=lm_labels, mc_token_ids=mc_token_ids, mc_labels=mc_labels)
    lm_loss, mc_loss = outputs.loss, outputs.mc_loss
    loss = lm_loss * lm_loss_coeff + mc_loss * mc_loss_coeff
    train_loss['lm_loss'] += lm_loss
    train_loss['mc_loss'] += mc_loss
    train_loss['total_loss'] += loss

    ### maybe I can include gradient accumulation conditions here. Gradient accumulation helps to fit larger batch size in memory ######
    ### With Gradient Accumulation will accompany gradient clipping #######
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    progressbar.update(1)
  

  for loss_type in train_loss:
    train_loss[loss_type] /= len(dataLoader)

  return train_loss

def evalStep(model, optimizer, dataLoader):

  loss = 0
  CEloss = 0
  precision = 0
  recall = 0
  valid_loss = {'lm_loss':0, 'mc_loss':0, 'total_loss': 0} 

  model.eval()
  with torch.no_grad():   
    for batch in dataLoader:
      input_ids, token_type_ids, lm_labels, mc_token_ids, mc_labels = batch
      outputs = model(input_ids = input_ids, token_type_ids = token_type_ids, labels=lm_labels, mc_token_ids=mc_token_ids, mc_labels=mc_labels)
      lm_logits, mc_logits = outputs.logits, outputs.mc_logits
      lm_loss, mc_loss = outputs.loss, outputs.mc_loss
      loss = lm_loss * lm_loss_coeff + mc_loss * mc_loss_coeff
      

      # print("lm_logits.shape: ", lm_logits.shape)
      # print("lm_labels.shape: ", lm_labels.shape)
      # print("mc_logits.shape: ", mc_logits.shape)
      # print("mc_labels.shape: ", mc_labels.shape)

      # lm_logits = lm_logits[0,0,:,:]
      # print("shape of the first of the batch's reply: ",lm_logits.shape)
      # encoded_tokens = torch.argmax(lm_logits, dim=-1)
      # encoded_tokens = encoded_tokens.view(-1) 
      # print(encoded_tokens.shape)

      # decoded_text = tokenizer.decode(encoded_tokens)
      # print(decoded_text)
      

      #### we calculate cross entropy loss to determine the difference between sentence label and the sentence output #####
      #### we use precision and recall to determine the difference between mc label and output
      CEloss += findCELoss(lm_logits, lm_labels)
      currRecall, currPrecision = findPrecisionAndRecall(mc_logits, mc_labels)
      recall += currRecall
      precision += currPrecision
      
      valid_loss['lm_loss'] += lm_loss
      valid_loss['mc_loss'] += mc_loss
      valid_loss['total_loss'] += loss

    for loss_type in train_loss:
      valid_loss[loss_type] /= len(dataLoader)

    return CEloss, precision, recall, valid_loss


In [ ]:
def trainAndEval(model, optimizer, trainDataLoader, validDataLoader):
  progressbar = tqdm(range(num_trainingSteps))

  for epoch in range(epochs):
    print("Epoch: ",epoch)
    
    train_loss = trainingStep(model, optimizer, trainDataLoader, progressbar)
    print("Training Loss: ")
    for loss_type in train_loss:
      print(loss_type,": ",train_loss[loss_type])
    print("\n")

    CEloss, precision, recall, valid_loss = evalStep(model, optimizer, validDataLoader)
    print("Validation Loss and Metrics:")
    print("CELoss: ", CEloss, " precision: ", precision, " recall: ",recall)
    for loss_type in valid_loss:
      print(loss_type,": ",valid_loss[loss_type])
    print("\n")
  

In [ ]:
trainAndEval(model, optimizer, trainDataLoader, validDataLoader)

  0%|          | 0/5625 [00:00<?, ?it/s]

Epoch:  0
Training Loss: 
lm_loss :  tensor(6.4646, device='cuda:0', grad_fn=<DivBackward0>)
mc_loss :  tensor(0.7026, device='cuda:0', grad_fn=<DivBackward0>)
total_loss :  tensor(7.1672, device='cuda:0', grad_fn=<DivBackward0>)


Validation Loss and Metrics:
CELoss:  tensor(65.1583, device='cuda:0')  precision:  19.99999994875  recall:  36.999999795
lm_loss :  tensor(5.9587, device='cuda:0')
mc_loss :  tensor(0.6931, device='cuda:0')
total_loss :  tensor(6.6518, device='cuda:0')


Epoch:  1
Training Loss: 
lm_loss :  tensor(5.4307, device='cuda:0', grad_fn=<DivBackward0>)
mc_loss :  tensor(0.7044, device='cuda:0', grad_fn=<DivBackward0>)
total_loss :  tensor(6.1350, device='cuda:0', grad_fn=<DivBackward0>)


Validation Loss and Metrics:
CELoss:  tensor(66.6038, device='cuda:0')  precision:  20.749999946874997  recall:  37.99999978833334
lm_loss :  tensor(5.8189, device='cuda:0')
mc_loss :  tensor(0.6934, device='cuda:0')
total_loss :  tensor(6.5123, device='cuda:0')


Epoch:  2
Train

In [ ]:
torch.save(model, driveRootPath + modelSavePath)

In [ ]:
if False:
  a = ['st','ri','ng']
  b = " ".join(a)
  print(b)

  d = tokenizer.encode('Komge')
  print(d)

  t = " ".join([tokenizer.decode(token) for token in d])
  print(t)













  

# Decoding and talking to the model

In [ ]:
def createDataset2(history, currentOutput, tokenizer, withEos = True):
  speaker1, speaker2, bos, eos = tokenizer.convert_tokens_to_ids(specialTokens[:-1])
  """
  1. we convert the history so that it includes speaker1, speaker2 and bos
  2. then we create input_ids, token_type_ids with it
  3. return them
  """  
  ### history should have several items in it. Each item is an array of tokens
  sequence = [[speaker2]+sentence if idx%2 else [speaker1]+sentence for idx, sentence in enumerate(history) ]
  sequence[0] = [bos] + sequence[0]
  sequence[-1] = sequence[-1] + ([eos] if withEos else [])

  input_ids = list(chain(*sequence))
  token_type_ids = [ speaker2 if idx%2 else speaker1 for idx, sentence in enumerate(sequence) for _ in range(len(sentence))]

  return input_ids, token_type_ids

In [ ]:
def top_filtering(logits, top_k=0., top_p=0.9, threshold=-float('Inf'), filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k, top-p (nucleus) and/or threshold filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k: <=0: no filtering, >0: keep only top k tokens with highest probability.
            top_p: <=0.0: no filtering, >0.0: keep only a subset S of candidates, where S is the smallest subset
                whose total probability mass is greater than or equal to the threshold top_p.
                In practice, we select the highest probability tokens whose cumulative probability mass exceeds
                the threshold top_p.
            threshold: a minimal threshold to keep logits
    """
    # print("logits shape: ", logits.shape)

    # if False:
    top_k = min(top_k, logits.size(-1))
    if top_k > 0:
        # Remove all tokens with a probability less than the last token in the top-k tokens
        top_logits, top_logits_indices = torch.topk(logits, top_k)
        # print("top_logits shape: ",top_logits.shape,"\ntop_logits_index shape: ",top_logits_indices.shape)
        indices_to_remove = logits < top_logits[..., -1, None]
        # print("indices_to_remove shape: ",indices_to_remove.shape)
        # print("sample row of indices to remove: ",indices_to_remove.shape[...,0])
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        # Compute cumulative probabilities of sorted tokens
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probabilities = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
        # print("cum sum prob shape: ",cumulative_probabilities.shape)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probabilities > top_p
        # print("indices_to_remove shape: ",sorted_indices_to_remove.shape)
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        # print("sample row of indices to remove: ",sorted_indices_to_remove[...,0])
        sorted_indices_to_remove[..., 0] = 0

        # Back to unsorted indices and set them to -infinity
        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value

    indices_to_remove = logits < threshold
    logits[indices_to_remove] = filter_value

    return logits

In [ ]:
def sample_sequence(history, tokenizer, model):
                    
    special_tokens_ids = tokenizer.convert_tokens_to_ids(specialTokens)
    current_output = []

    for i in range(max_output_length):
        input_ids, token_type_ids = createDataset2(history, current_output, tokenizer, withEos=False)
        # print("input_ids: ",input_ids,"\ntoken_type_ids: ",token_type_ids)      
        
        input_ids = torch.tensor(input_ids).to(device).unsqueeze(0)
        token_type_ids = torch.tensor(token_type_ids).to(device).unsqueeze(0)

        outputs = model(input_ids = input_ids, token_type_ids= token_type_ids)
        logits = outputs.logits
        # print("logits.shape: ",logits.shape)

        
        # makes the softmax scores bigger. Their sum will exceed 1. But if we choose to use top_p, we would choose fewer options because of using temparature.
        # If we didn't use the temparature, the probabilities would be smaller which would lead to smaller cum sum. Meaning, it will take many probabilites scores to add to a small cumulative probability
        # Then we would end up choosing a lot of options, if not almost all
        encoded_text = torch.argmax(logits[0], dim =-1)
        # print("decoded text: "," ".join([tokenizer.decode(token) for token in encoded_text]) )
        logits = logits[0, 0, :] / temperature 
        # print("logits.shape after dividing with temparature: ",logits.shape)
        # if False:
        logits = top_filtering(logits, top_k=top_k, top_p=top_p)
        probs = F.softmax(logits, dim=-1)

        prev = torch.multinomial(probs, 1)
        if i < min_output_length and prev.item() in special_tokens_ids:
            while prev.item() in special_tokens_ids:
                if probs.max().item() == 1:
                    print("Warning: model generating special token with probability 1.")
                    break  # avoid infinitely looping over special token
                prev = torch.multinomial(probs, num_samples=1)

        if prev.item() in special_tokens_ids:
            break
        current_output.append(prev.item())

    return current_output

In [ ]:
def run():
	
  if random_seed != 0:
    random.seed(random_seed)
    torch.random.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)

  tokenizer_class, model_class = (GPT2TokenizerFast, GPT2LMHeadModel) if model_type == 'gpt2' else (OpenAIGPTTokenizerFast, OpenAIGPTLMHeadModel)
  # tokenizer = tokenizer_class.from_pretrained(model_type)
  
  model = torch.load(driveRootPath+modelSavePath)
  model.to(device)
  model.resize_token_embeddings(len(tokenizer))
  
  # print("tokenizer size: ", len(tokenizer))
  # print(tokenizer.convert_tokens_to_ids(specialTokens[:-1]))
  # print(tokenizer.encode("wut do"))


  history = []
  while True:
      raw_text = input(">>> ")
      while not raw_text:
          print('Prompt should not be empty!')
          raw_text = input(">>> ")
      history.append(tokenizer.encode(raw_text))
      with torch.no_grad():
          out_ids = sample_sequence(history, tokenizer, model)
      history.append(out_ids)
      history = history[-(2*pairOfExchanges+1):]
      out_text = tokenizer.decode(out_ids, skip_special_tokens=True)
      print(out_text)


In [ ]:
random_seed = 0
temperature = 0.7
top_k = 0
top_p = 0.9
max_output_length = 20
min_output_length = 1
model_type = 'openai-gpt'


In [ ]:
run()

>>> hello
?, boltesos? kortesi,? hoiche?'eddinfalateto AbarratargularehEtaiSGDmushkileddin
>>> wut do
?? shara,???! chash,?'chash???,,??
